<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/01-Mi-segundo-modelo-Plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Mi primer modelo
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Nombre y Apellido &bull; Fecha
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

Este notebook está diseñado parcticar en el entrenamiento y evaluación de un modelo de regresión lineal utilizando datos reales. Los objetivos principales son:

- Entrenar un modelo de regresión lineal.
- Evaluar el ajuste del modelo.


Del conjunto de datos disponible [aquí](https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv), se desea predecir la variable `price` en función de las variables `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`.

In [2]:
# Paquetes necesarios
import numpy as np  # Para generación de datos y operaciones numéricas
import pandas as pd  # Para manejo de datos estructurados
import matplotlib.pyplot as plt  # Para visualización de datos
from sklearn.linear_model import LinearRegression  # Para el modelo de regresión lineal
from sklearn.metrics import mean_squared_error, r2_score  # Para evaluación del modelo
from sklearn.model_selection import train_test_split

---
## <font color='264CC7'> Lectura de Datos </font>


Realiza la lectura de los datos y presenta las primeras filas del conjunto de datos.

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv')
df_modelo = pd.DataFrame(df, columns=['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors'])
df_modelo.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors
0,221900.0,3,1.00,1180,5650,1.0
1,538000.0,3,2.25,2570,7242,2.0
2,180000.0,2,1.00,770,10000,1.0
3,604000.0,4,3.00,1960,5000,1.0
4,510000.0,3,2.00,1680,8080,1.0


---
## <font color='264CC7'> Preprocesamiento de datos </font>

Realiza una normalización de los datos, prueba diferentes técnicas y selecciona la que mejor se ajuste a los datos.

In [ ]:
#Revisamos los tipos correctos de los datos
df_modelo.dtypes
#Normalizamos de acuerdo a diferentes técnicas. Antes de eso vemos un summary de cada variable
print(df_modelo.describe())

In [17]:

#Los datos muestran una desviación estándar en price, sqft_living y sqft_lot muy grande. Sin embargo a pesar de que la estandarización se ve como buena opción
#Los datos de precios y los de espacio de vivienda y lot no tienen sentido el presentarlos negativos ya que no tiene sentido en el caso práctico. Por tanto,
#y viendo que por tener un mínimo distinto de 0 que no es normalmente reconocido es importante usar para estos tres una normalización min-max.
#Price sqft living y sqft lot:
priceminmax = (df_modelo["price"] - df_modelo["price"].min()) / (df_modelo["price"].max() - df_modelo["price"].min())
sqft_livingminmax = (df_modelo["sqft_living"] - df_modelo["sqft_living"].min()) / (df_modelo["sqft_living"].max() - df_modelo["sqft_living"].min())
sqft_lotminmax = (df_modelo["sqft_lot"] - df_modelo["sqft_lot"].min()) / (df_modelo["sqft_lot"].max() - df_modelo["sqft_lot"].min())
print (f"price: ",priceminmax.head(), sqft_livingminmax.head(), sqft_lotminmax.head())

price: 0    0.019266
1    0.060721
2    0.013770
3    0.069377
4    0.057049
Name: price, dtype: float64 0    0.067170
1    0.172075
2    0.036226
3    0.126038
4    0.104906
Name: sqft_living, dtype: float64 0    0.003108
1    0.004072
2    0.005743
3    0.002714
4    0.004579
Name: sqft_lot, dtype: float64


---
## <font color='264CC7'> Características y etiquetas </font>

Separa las características y las etiquetas y mostrar la dimensión de cada conjunto.

---
## <font color='264CC7'> Conjunto de entrenamiento y prueba </font>

Divide los datos en dos conjuntos: uno para entrenamiento y otro para prueba y visualiza la dimensión de cada conjunto. Usar `random_state` igual al últmo número de tu cédula.

Revisa la distribución de las etiquetas en los conjuntos de entrenamiento y prueba.

---
## <font color='264CC7'> Entrenamiento </font>

Define un modelo de regresión lineal y lo entrena con los datos de entrenamiento.

---
## <font color='264CC7'> Evaluacion </font>

Realiza la predicciones con el conjunto de prueba y muestra los valores en una tabla.

Calcula el error cuadrático medio.

Escribe el error cuadrático medio para cada tipo de normalización:

- Normalización MaxAbsScaler:
- Normalización MinMaxScaler:
- Normalización RobustScaler:
- Normalización StandardScaler:
- Normalización Normalizer:
- Normalización QuantileTransformer:
- Normalización PowerTransformer:

Escribe un párrafo sobre qué hace normalización no vista en clase:

- **Normalización ...:** ...

---
## <font color='264CC7'> Cross Validation </font>

Realiza validación cruzada con 5 particiones y muestra el error cuadrático medio promedio con la normalización que mejor se ajuste a los datos para confirmar que el modelo no está sobreajustado.